In [1]:
%matplotlib inline

import pandas as pd
import seaborn as sns
import numpy as np
import matplotlib.pyplot as plt
import _pickle as cPickle
import nltk

In [3]:
df = pd.read_csv('lsapp.tsv', sep='\t')
df_open = df.loc[df['event_type'] == 'Opened']

In [6]:
df_open.loc[:,'timestamp'] = pd.to_datetime(df_open['timestamp'])
df.loc[:,'timestamp'] = pd.to_datetime(df['timestamp'])

/home/maliann/anaconda3/lib/python3.7/site-packages/pandas/core/indexing.py:494: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[item] = s


In [8]:
df

,user_id,session_id,timestamp,app_name,event_type
0,0,1,2018-01-16 06:01:05,Minesweeper Classic (Mines),Opened
1,0,1,2018-01-16 06:01:05,Minesweeper Classic (Mines),Closed
2,0,1,2018-01-16 06:01:07,Minesweeper Classic (Mines),Opened
3,0,1,2018-01-16 06:01:07,Minesweeper Classic (Mines),Closed
4,0,1,2018-01-16 06:01:08,Minesweeper Classic (Mines),Opened
...,...,...,...,...,...
3658584,291,76247,2018-04-06 14:35:15,Facebook,Closed
3658585,291,76247,2018-04-06 14:35:15,Facebook,Opened
3658586,291,76247,2018-04-06 14:35:37,Facebook,Closed
3658587,291,76247,2018-04-06 14:35:37,Facebook,Opened


In [10]:
df['time_dff']  = df[['timestamp']].diff()
df['interaction_id'] = (((df.timestamp-df.timestamp.shift(1) > pd.Timedelta(1, 'm')) 
                              & (df.event_type == 'Opened')) 
                              | ~(df.app_name == df.app_name.shift(1))
                              | ~(df.user_id == df.user_id.shift(1))).cumsum()
df['session_id'] = (((df.timestamp-df.timestamp.shift(1) > pd.Timedelta(5, 'm')) 
                              & (df.event_type == 'Opened')) 
                              | ~(df.user_id == df.user_id.shift(1))).cumsum()
df_start = df.drop_duplicates(subset=['interaction_id'], keep='first')
df_end = df.drop_duplicates(subset=['interaction_id'], keep='last')
df_start.set_index('interaction_id', inplace=True)
df_end.set_index('interaction_id', inplace=True)
df_start.loc[:, 'open_time'] = df_start['timestamp']
df_start.loc[:, 'close_time']  = df_end['timestamp']

/home/maliann/anaconda3/lib/python3.7/site-packages/pandas/core/indexing.py:376: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[key] = _infer_fill_value(value)


In [11]:
df_start

,user_id,session_id,timestamp,app_name,event_type,time_dff,open_time,close_time
interaction_id,,,,,,,,
1,0,1,2018-01-16 06:01:05,Minesweeper Classic (Mines),Opened,NaT,2018-01-16 06:01:05,2018-01-16 06:01:09
2,0,1,2018-01-16 06:03:44,Minesweeper Classic (Mines),Opened,00:02:35,2018-01-16 06:03:44,2018-01-16 06:04:17
3,0,1,2018-01-16 06:25:54,Gmail,User Interaction,00:21:37,2018-01-16 06:25:54,2018-01-16 06:25:54
4,0,1,2018-01-16 06:26:05,Google,Opened,00:00:11,2018-01-16 06:26:05,2018-01-16 06:26:10
5,0,1,2018-01-16 06:26:10,Instagram,Opened,00:00:00,2018-01-16 06:26:10,2018-01-16 06:26:21
...,...,...,...,...,...,...,...,...
599631,291,33218,2018-04-06 12:47:28,Facebook Messenger,Opened,00:03:29,2018-04-06 12:47:28,2018-04-06 12:53:13
599632,291,33219,2018-04-06 13:20:12,Settings,Opened,00:26:59,2018-04-06 13:20:12,2018-04-06 13:20:14
599633,291,33220,2018-04-06 14:34:15,Settings,Opened,01:14:01,2018-04-06 14:34:15,2018-04-06 14:34:17
